## Lennard-Jones Potential
<br>


$$V(r)=4\epsilon\big[\big(\frac{\delta}{r}\big)^{12}-\big(\frac{\delta}{r}\big)^6\big]$$
<br>
<br>
Suppose we have $m$ molecules whose positions are denoted by $\textbf{a}_m$ that exist in $\mathbb{R^n}$. Let $\textbf{A}$ be an $m\times{n}$ matrix that contains the molecules, and let $\textbf{r}$ be a $m\times{m}$ square symetric matrix where $r_{ij}=\|\textbf{A}_i-\textbf{A}_j\|$ 
<br>
<br>
$$
\begin{align*}
\textbf{A}&=\left[\begin{matrix} \textbf{a}_1\\\textbf{a}_2\\\vdots\\\textbf{a}_m\end{matrix}\right]=
\left[\begin{matrix} a_{1_1} & a_{1_2} & \dots & a_{1_n}\\ a_{2_1} & \ddots & & \\\vdots & & \ddots &\\a_{m_1} & & & a_{m_n}\end{matrix}\right]\\
\textbf{r}&=\left[\begin{matrix} \|\textbf{A}_1-\textbf{A}_1\| & \dots & \|\textbf{A}_1-\textbf{A}_j\| \\ \vdots &\ddots & \\ \|\textbf{A}_i-\textbf{A}_1\| & & \|\textbf{A}_m-\textbf{A}_m\| \end{matrix}\right]
\end{align*}
$$
<br>
<br>
The total energy $E$ of a system $\textbf{A}$ with a distance matrix $\textbf{r}$ is defined as the sum of the LJ energy between every molecule.
<br>
<br>
$$E(\textbf{A})=\sum_{i=0}^{m-1}\sum_{j=i+1}^{m}V(r_{ij})\quad {\scriptsize \textrm{in total, } {m\choose 2 } {\textrm{ terms. (upper triangle of }}\textbf{r})}
$$

For computational purposes, only the upper triangle of $\textbf{r}$ is calculated, and it is flattened to 1 dimension. The final energy calculation then only needs one iterator over the span of $\textbf{r}$

In [1]:
import numpy as np


def Energy(A,dim):
    def V(r,epsilon=1,delta=1):#Returns the potential between two molecules of distance r
        return 4*epsilon*((delta/r)**12 - (delta/r)**6)
    
    def r(A):#Returns a distance matrix. Only the upper triangle needs to be calculated; the rest can be 0
        r=[]
        for i in range(0,A.shape[0]-1):
            for j in range(i+1,A.shape[0]):
                r.append(np.sqrt(np.dot(A[i]-A[j],A[i]-A[j])))
        return np.asarray(r)
    A=np.reshape(A,(int(len(A)/dim),dim))
    r1=r(A)
    total=0.
    for i in range(len(r1)):
            total+=V(r1[i])
    return total.flatten()

## Lennard Jones Force Function

<br>

Lets examine the force vector $\textbf{F}$ on a location $\textbf{x}$ generated from a particle $\textbf{a}$ in $\mathbb{R^n}$ space. Let $(x_1,x_2,\dots,x_n)$ denote the basis vectors. <br>
For simplicity,  $\epsilon,\delta$ =1.
<br>
<br>
$$
\begin{align*}
\textbf{F}(\textbf{x})&=\nabla V\\
&=4\nabla\big[\big(\frac{1}{r}\big)^{12}-\big(\frac{1}{r}\big)^6\big]\\
&= \big[-\frac{48}{r^{13}}+\frac{24}{r^7}\big]\nabla r\\
&= \big[-\frac{48}{r^{13}}+\frac{24}{r^7}\big]\big(\frac{\partial}{\partial x_1}\hat x_1+\frac{\partial}{\partial x_2}\hat x_2+\dots+\frac{\partial}{\partial x_n}\hat x_n\big)\big((a_1-x_1)^2+(a_2-x_2)^2+\dots+(a_n-x_n)^2\big)^{\frac{1}{2}}\\
&= \big[\frac{48}{r^{14}}-\frac{24}{r^8}\big]\big((a_1-x_1)\hat x_1+(a_2-x_2)\hat x_2 + \dots + (a_n-x_n)\hat x_n\big)\\
&= \big[\frac{48}{r^{14}}-\frac{24}{r^8}\big](\textbf{a}-\textbf{x})
\end{align*}
$$
<br>
<br>
Due to the principle of superposition, we can easily extrapolate to include $m$ molecules
<br>
<br>
$$\textbf{F}(\textbf{x})=\sum_{i=1}^{m}\big[\frac{48}{r_i^{14}}-\frac{24}{r_i^8}\big](\textbf{a}_i-\textbf{x})
$$
<br>
<br>
If this force function was evaluated at every particle location $\textbf{a}_m$, you would have a complete force matrix. The direction of greatest increase for each particle would be specifically calculated based on the energy of the system $\textbf{A}$, and each location parameter $a_{mn}$ would be automatically filled in by the gradient. Thus,

$$\nabla E=\left[\begin{matrix}\textbf{F}(\textbf{a}_1) \\ \textbf{F}(\textbf{a}_2) \\ \vdots \\ \textbf{F}(\textbf{a}_m)\end{matrix}\right]
$$
<br>
<br>
Within the evaluation of $\textbf{F}$, the contribution of a particle onto itself must manually removed, or else $r$ calculation will yield a divide by 0.

In [2]:
def gradientE(A,dim):
    def F(A,x):
        total=np.zeros([len(x)])
        for i in range(A.shape[0]):
            r=np.sqrt(np.dot(A[i]-x,A[i]-x))
            k=(48/r**14)-(24/r**8)
            total+=k*(A[i]-x)
        return total

    A=np.reshape(A,(int(len(A)/dim),dim))
    force=np.zeros(A.shape)
    for i in range(A.shape[0]):
        A1=A.copy()
        A1=np.delete(A1,i,0)
        force[i]=F(A1,A[i])

    return force.flatten()

In [3]:
#Accuracy Check
from LJ import LJ,LJ_force
import numpy as np
def init(m,n):#Returns a random, normalized array of size mxn
    A=np.random.uniform(low=-1.,high=1.,size=(m,n))
    return A
def error(x,y): #I'm using the LJ from your optimization repository as the theoretical
    return (100*(x-y)/x)
energy=0.
gradient=0.
for i in range(100):#Average the error over 100 random systems of A
    A=init(3,3)
    energy+=np.abs(error(Energy(A.flatten(),3),LJ(A.flatten(),3)))
    gradient+=np.abs(error(np.linalg.norm(gradientE(A.flatten(),3)),np.linalg.norm(LJ_force(A.flatten(),3))))
energy=float(energy/100.)
graident=gradient/100
print('Percentage Error of LJ-Potential: ',energy)
print('Percentage Error of LJ-Force: ', gradient)   

Percentage Error of LJ-Potential:  1.0015150846790183e-13
Percentage Error of LJ-Force:  3.5673908814841597e-12


## Conjugate Gradient for Non-Linear Systems
<br>

$\text{Algorithm detailed in }\small\textit{Numerical Optimization} \text{ by Nocedal & Wright.}$
### Conjugate Gradient
$\text{Given } x_0;$<br>
$\text{Evalute } f_0=f(x_0), \nabla f_0= \nabla f(x_0);$<br>
$\text{Set }p_0=- \nabla f_0, k\leftarrow0$<br>
$\begin{align*} \textbf{while } \nabla &f_k \neq 0 \\
&\text{Compute } \textbf{LineSearch}(\alpha_k) \text{ and set }x_{k+1} = x_k + \alpha_kp_k;\\
&\text{Evalute } \nabla f_{k+1}; \\
\beta_{k+1}^{\text{PR}} &\leftarrow \text{max}\left(\frac{(\nabla f_{k+1}-\nabla f_{k})^T \nabla f_{k+1}}{\nabla f_k^T \nabla f_k},0\right);\\
p_{k+1} &\leftarrow -\nabla f_{k+1} + \beta_{k+1}^\text{PR}p_k;\\
k &\leftarrow k + 1
\end{align*}$
<br>
<br>

In [7]:
import numpy as np

"""
Parameters
-----------
f : function to be evaluated. First argument must be x0
x0 : initial guess of parameters begin the iterative algorithm
     should be preconditioned to not be in an ill-behaving region of f for ideal performance
fprime : analytical gradient of f. If none is given defaults to using a two point finite difference method
args : arguments to pass into f and fprime
gtol : tolerance of the largest derivative of all the parameters before the algorithm has converged to local minimum
maxiter : maximum number of steps the algorithm will take
eps : step sized used calculate the finite difference method if no analytical gradient is given


Returns [fval,xk,gnorm]
----------------------
fval : the minimum value of the function
xk : the minimizer of f
gnorm : the largest partial derivative value of all the parameters
"""

def CG(f,x0,fprime=None,args=(),gtol=1e-5,maxiter=None,eps = np.sqrt(np.finfo(float).eps)):

    
    def approx_jac(A,dim): #replaces fprime if none is given. Uses two point finite difference method
        fprime=np.zeros([len(A)])
        for i in range(len(A)):
            delta=np.zeros([len(A)])
            delta[i]=eps
            fprime[i]=(f(A+delta)-f(A-delta))/(2*eps)
        return fprime
    
    
    def wrap_function(function, args):#wraps arguments into the function
        ncalls = [0]
        def function_wrapper(*wrapper_args):
            ncalls[0] += 1
            return function(*(wrapper_args + args))


        return ncalls, function_wrapper
    
    
    func_calls, f = wrap_function(f, args)
    if fprime==None:
        grad_calls,fprime=wrap_function(approx_jac,args)
    else:
        grad_calls,fprime=wrap_function(fprime, args)
    
    
    if maxiter is None:
        maxiter = len(x0) * 200 #The artificial maximum number of iterations

    xk=x0
    gfk=fprime(xk)
    pk=-gfk
    old_fval = f(xk) #Artificially generates previous steps to help the line search run smoother
    old_old_fval = old_fval + np.linalg.norm(gfk) / 2
    k=0
    gnorm = np.amax(np.abs(gfk))
    cached_step = [None]#holds caluclated values to prevent redundant computation
    
    
    while(gnorm>gtol) and (k<maxiter):
        
        
        def step(alpha,gfk1=None):#A step length using the polak-ribiere beta value
            xk1 = xk + alpha * pk
            if gfk1 is None:
                gfk1=fprime(xk1)
            betak1 = max(0, np.dot((gfk1-gfk), gfk1) / np.dot(gfk,gfk))
            pk1 = -gfk1 + betak1 * pk
            gnorm = np.amax(np.abs(gfk1))
            return (alpha, xk1, pk1, gfk1, gnorm)
        
        
        def descent_condition(alpha, gfk1=None):#Using the polak-ribiere step length, pk is not guarenteed to satisfy
            cached_step[:] = step(alpha, gfk1)  #sufficient decrease condition. Must test for it.
            alpha, xk, pk, gfk,gnorm=step(alpha,gfk1)
            if gnorm<gtol:
                return True
            return np.dot(pk, gfk) <= -.01 * np.dot(gfk, gfk)

        
        try:
            alpha_k, fc, gc, old_fval, old_old_fval, gfk1 = \
                                    LineSearchHub(f, fprime, xk, pk, gfk, old_fval,
                                          old_old_fval, c2=0.4, amin=1e-100, amax=1e100,
                                          extra_condition=descent_condition)
        except _LineSearchError:
            break


        if alpha_k == cached_step[0]:
            alpha_k, xk, pk, gfk, gnorm = cached_step
        else:
            alpha_k, xk, pk, gfk, gnorm = step(alpha_k, gfk1)
        k += 1
    
    fval=old_fval
    return[fval,xk,gnorm]

In [ ]:
import numpy as np
from scipy.optimize import minpack2
from scipy.optimize.linesearch import line_search_wolfe1
import warnings
from scipy.optimize.linesearch import LineSearchWarning

class _LineSearchError(RuntimeError):
    pass

    
"""
LineSearchHub Parameters
------------------------
f : function to be evaluated. First argument must be x0
fprime : gradient of f
xk= the fixed point on the line to be minimized
pk : the direction of the line
gfk : fprime evaluated at xk
old_fval : function value at previous point
old_old_fval : function value at previous previous point
c2 : constant for determining curvature condition
amin;amax : min and max alpha values
extra_condition : required condition that the new alpha must satsify. Has arguments of form (alpha, gfkp1)
                  where gfkp1 is the gradient at the new point proposed by alpha


Returns [fval,xk,gnorm]
----------------------
fval : the minimum value of the function
xk : the minimizer of f
gnorm : the largest partial derivative value of all the parameters
""" 
    
    
def LineSearchHub(f, fprime, xk, pk, gfk, old_fval, old_old_fval,
                     **kwargs):


    extra_condition = kwargs.pop('extra_condition', None)#the 1st line search method doesn't have an extra condition as a parameter

    ret = line_search_wolfe1(f, fprime, xk, pk, gfk,
                             old_fval, old_old_fval,
                             **kwargs)

    if ret[0] is not None and extra_condition is not None:
        xp1 = xk + ret[0] * pk
        if not extra_condition(ret[0],ret[5]):
            ret = (None,)

    if ret[0] is None:#This is the pure pythonic line search that is used if 1st method fails to converge.
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', LineSearchWarning)
            kwargs2 = {}
            for key in ('c1', 'c2', 'amax'):
                if key in kwargs:
                    kwargs2[key] = kwargs[key]
            ret = search(f, fprime, xk, pk, gfk,
                                     old_fval, old_old_fval,
                                     extra_condition=extra_condition,
                                   **kwargs2)



    if type(ret) is not tuple:
        raise _LineSearchError()
    if ret[0] is None:
        raise _LineSearchError()
    return ret
    
def zoom(a_lo, a_hi, phi_lo, phi_hi, derphi_lo,
              phi, derphi, phi0, derphi0, c1, c2, extra_condition): #Given an interval of alpha values, finds the minimum
        maxiter = 100
        i = 0
        delta1 = 0.2  # cubic interpolant check
        delta2 = 0.1  # quadratic interpolant check
        phi_rec = phi0
        a_rec = 0
        while True:
#             print([a_lo,a_hi])
            # interpolate to find a trial step length between a_lo and
            # a_hi Need to choose interpolation here.  Use cubic
            # interpolation and then if the result is within delta *
            # dalpha or outside of the interval bounded by a_lo or a_hi
            # then use quadratic interpolation, if the result is still too
            # close, then use bisection

            dalpha = a_hi - a_lo
            if dalpha < 0:#Orders the interval boundaries
                a, b = a_hi, a_lo
            else:
                a, b = a_lo, a_hi


            if (i > 0):#Cannot lead with a cubic interpolation because the formula requires a previous iteration.
                cchk = delta1 * dalpha #The distance the minimum must be from either endpoint (20% of the range)
                a_j = _cubicmin(a_lo, phi_lo, derphi_lo, a_hi, phi_hi,
                                a_rec, phi_rec)
                
            if (i == 0) or (a_j is None) or (a_j > b - cchk) or (a_j < a + cchk): #runs in 1st iteration, or if cubic fails
                qchk = delta2 * dalpha #quadratic interpolation shouldn't be within 10% of the range from the boundaries
                a_j = _quadmin(a_lo, phi_lo, derphi_lo, a_hi, phi_hi)
                if (a_j is None) or (a_j > b-qchk) or (a_j < a+qchk):
                    a_j = a_lo + 0.5*dalpha  #initializes a_j for a golden section method if both cubic and quadratic fail


            phi_aj = phi(a_j)
            if (phi_aj > phi0 + c1*a_j*derphi0) or (phi_aj >= phi_lo): #Tests Wolfe conditions 1. If fails, will reset boundaries
                phi_rec = phi_hi                                       #and try again
                a_rec = a_hi
                a_hi = a_j
                phi_hi = phi_aj
            else:
                derphi_aj = derphi(a_j) #Tests Wolfe conditions 2. If passes, a_j is a good step length, loop breaks.
                if abs(derphi_aj) <= -c2*derphi0 and extra_condition(a_j,None):
                    a_star = a_j
                    val_star = phi_aj
                    valprime_star = derphi_aj
                    break
                if derphi_aj*(a_hi - a_lo) >= 0:
                    phi_rec = phi_hi
                    a_rec = a_hi
                    a_hi = a_lo
                    phi_hi = phi_lo
                else:
                    phi_rec = phi_lo
                    a_rec = a_lo
                a_lo = a_j
                phi_lo = phi_aj
                derphi_lo = derphi_aj
            i += 1
            if (i > maxiter):
                a_star = None
                val_star = None
                valprime_star = None
                break
        return a_star, val_star, valprime_star


def _cubicmin(a, fa, fpa, b, fb, c, fc): #Uses the known minimum of a cubic polynomial that passes through the boundaries. 
                                         #Requires a previous step

    # f(x) = A *(x-a)^3 + B*(x-a)^2 + C*(x-a) + D

    with np.errstate(divide='raise', over='raise', invalid='raise'):
        try:
            C = fpa
            db = b - a
            dc = c - a
            denom = (db * dc) ** 2 * (db - dc)
            d1 = np.empty((2, 2))
            d1[0, 0] = dc ** 2
            d1[0, 1] = -db ** 2
            d1[1, 0] = -dc ** 3
            d1[1, 1] = db ** 3
            [A, B] = np.dot(d1, np.asarray([fb - fa - C * db,
                                            fc - fa - C * dc]).flatten())
            A /= denom
            B /= denom
            radical = B * B - 3 * A * C
            xmin = a + (-B + np.sqrt(radical)) / (3 * A)
        except ArithmeticError:
            return None
    if not np.isfinite(xmin):
        return None
    return xmin    
    
    
    
def _quadmin(a, fa, fpa, b, fb): #Uses the known minium of a quadratic poylnomial that passes through the boundaries.
    
    # f(x) = B*(x-a)^2 + C*(x-a) + D
    with np.errstate(divide='raise', over='raise', invalid='raise'):
        try:
            D = fa
            C = fpa
            db = b - a * 1.0
            B = (fb - D - C * db) / (db * db)
            xmin = a - C / (2.0 * B)
        except ArithmeticError:
            return None
    if not np.isfinite(xmin):
        return None
    return xmin

    
def search(f,fprime,xk,pk,gfk,old_fval,old_old_fval,c1=1e-4, c2=0.4, amin=1e-100, amax=1e100, maxiter=100, extra_condition=None):
    #tries to establish an interval containing the minimizer. If found, calls on the zoom function to return the best step size
    fc=0
    gc=0
    def phi(alpha):
        if alpha!=None:
            return f(xk+alpha*pk)
        else:
            return None
    
    def phiprime(alpha):
        return np.dot(fprime(xk+alpha*pk),pk)



    if extra_condition is None:
        def extra_condition(a,b):
            return True

    alpha=np.zeros([2])
    if phiprime(0.)!=0:
        alpha[1]=min(1.,1.01*2*(old_fval-old_old_fval)/phiprime(0.))#Ensures superlinear convergence rate 
    else:
        alpha[1]=1.
    if alpha[1]<0.:
        alpha[1]=1.
    i=0
    phiprime0=phiprime(0.)
    phi0=phi(0.)
    
    while i<maxiter:
 
        phik=phi(alpha[1])
        if ((phik>(phi0+c1*alpha[1]*phiprime0)) or ((phik >=phi(alpha[0])) and i>1)):#tests wolfe conditions 1
            alphastar=zoom(alpha[0],alpha[1],phi(alpha[0]),phi(alpha[1]),phiprime(alpha[0]),phi,phiprime,phi0,phiprime0,c1,c2,extra_condition)[0]
            return alphastar,fc,gc,phi(alphastar),old_fval,None

        phikprime=phiprime(alpha[1]) #tests wolfe conditions 2
        if np.abs(phikprime)<=-c2*phiprime0:
            if extra_condition(alpha[1],None):
                alphastar=alpha[1]
                return alphastar,fc,gc,phi(alphastar),old_fval,None

        if phikprime>=0: #if the gradient of the new test alpha is positive and failed wolfe 2, it means the minimizer is in the range, but the boundaries are flipped
            alphastar=zoom(alpha[1],alpha[0],phi(alpha[1]),phi(alpha[0]),phiprime(alpha[1]),phi,phiprime,phi0,phiprime0,c1,c2,extra_condition)[0]
            return alphastar,fc,gc,phi(alphastar),old_fval,None
        alpha2=1.1*alpha[1]
        alpha[0]=alpha[1]
        alpha[1]=alpha2
        i+=1

    
    
    
    
    


    


In [12]:
# Time Test
import matplotlib.pyplot as plt
import time

t1=[]
t2=[]
dim=3
nmin=2
nmax=20
for i in np.arange(nmin,nmax,1):
    A=init(i,dim)
    start=time.time()
    CG(Energy,A.flatten(),args=(dim,))
    end=time.time()
    t1.append(end-start)
    start=time.time()
    CG(Energy,A.flatten(),args=(dim,),fprime=gradientE)
    end=time.time()
    t2.append(end-start)
t1=np.asarray(t1)
t2=np.asarray(t2)




x=np.arange(nmin,nmax,1)
fig=plt.figure()
ax1=fig.add_subplot(111)
ax1.plot(x,t1/t2)#The y axis is the speed of the approximated gradient divided by the speed of the analytical gradient.
ax1.set_xlabel('Number of Molecules')
ax1.set_ylabel('Speed Ratio')
ax1.set_title('My CG')
plt.show()
print('With the analytical gradient, the speed is approximately 2n faster where n is the number of molecules')

In [22]:
import scipy.optimize
import myscipy_optimize
q=0
t=0
n=1000
for i in range(n):
    A=init(3,3).flatten()
    while(Energy(A,3)>100):
        A=init(3,3).flatten()
    if CG(Energy,A,fprime=gradientE,args=(3,))[0]<-2.99:
        q+=1
    if scipy.optimize.minimize(Energy,A,jac=gradientE,args=(3,),method='cg').fun<-2.99:
        t+=1
print(q/n)
print(t/n)


0.999
0.999
